# Language Agent Tree Search

<a href="https://colab.research.google.com/github/run-llama/llama_index/blob/main/llama-index-packs/llama-index-packs-lats/examples/language_agent_tree_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[LATS (Language Agent Tree Search)](https://arxiv.org/pdf/2310.04406.pdf) by Zhou et al. combines LLM capabilities in planning, acting, and reasoning within a Monte Carlo tree search framework, allowing for deliberate and adaptive problem-solving guided by external feedback and self-reflection.

We've implemented this agent as a LlamaPack - you can either pip install it to run it out-of-the-box or call `download_llama_pack` to load the pack.

## Setup

In [ ]:
%pip install llama-index-packs-agents-lats
%pip install llama-index-llms-openai
%pip install llama-index-embeddings-openai
%pip install llama-index-core llama-index-readers-file

### Define Global Settings

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = "sk-..."

import nest_asyncio

nest_asyncio.apply()

In [ ]:
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import Settings

# NOTE: a higher temperate will help make the tree-expansion more diverse
llm = OpenAI(model="gpt-4-turbo", temperature=0.6)
embed_model = OpenAIEmbedding(model="text-embedding-3-small")

Settings.llm = llm
Settings.embed_model = embed_model

### Download Data

In [ ]:
!mkdir -p 'data/10k/'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/uber_2021.pdf' -O 'data/10k/uber_2021.pdf'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/lyft_2021.pdf' -O 'data/10k/lyft_2021.pdf'

In [ ]:
import os
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    load_index_from_storage,
)
from llama_index.core.storage import StorageContext


if not os.path.exists("./storage/lyft"):
    # load data
    lyft_docs = SimpleDirectoryReader(
        input_files=["./data/10k/lyft_2021.pdf"]
    ).load_data()
    uber_docs = SimpleDirectoryReader(
        input_files=["./data/10k/uber_2021.pdf"]
    ).load_data()

    # build index
    lyft_index = VectorStoreIndex.from_documents(lyft_docs)
    uber_index = VectorStoreIndex.from_documents(uber_docs)

    # persist index
    lyft_index.storage_context.persist(persist_dir="./storage/lyft")
    uber_index.storage_context.persist(persist_dir="./storage/uber")
else:
    storage_context = StorageContext.from_defaults(
        persist_dir="./storage/lyft"
    )
    lyft_index = load_index_from_storage(storage_context)

    storage_context = StorageContext.from_defaults(
        persist_dir="./storage/uber"
    )
    uber_index = load_index_from_storage(storage_context)

### Setup Tools

In [ ]:
lyft_engine = lyft_index.as_query_engine(similarity_top_k=3)
uber_engine = uber_index.as_query_engine(similarity_top_k=3)

In [ ]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata

query_engine_tools = [
    QueryEngineTool(
        query_engine=lyft_engine,
        metadata=ToolMetadata(
            name="lyft_10k",
            description=(
                "Provides information about Lyft financials for year 2021. "
                "Use a detailed plain text question as input to the tool. "
                "The input is used to power a semantic search engine."
            ),
        ),
    ),
    QueryEngineTool(
        query_engine=uber_engine,
        metadata=ToolMetadata(
            name="uber_10k",
            description=(
                "Provides information about Uber financials for year 2021. "
                "Use a detailed plain text question as input to the tool. "
                "The input is used to power a semantic search engine."
            ),
        ),
    ),
]

## Setup Agent

Now we can setup the LATS agent.

Here, `num_expansions` refers to the number of possible sub-actions to explore under each node. `num_expansions=2` means we will explore to possible next-actions for every parent action.

`max_rollouts` refers to how deep each exploration of the search space continues. `max_rollouts=5` means a maximum depth of 5 is explored in the tree.

In [ ]:
from llama_index.core.agent import AgentRunner
from llama_index.packs.agents_lats import LATSAgentWorker


agent_worker = LATSAgentWorker.from_tools(
    query_engine_tools,
    llm=llm,
    num_expansions=2,
    max_rollouts=10,  # using -1 for unlimited rollouts
    verbose=True,
)
agent = AgentRunner(agent_worker)

## Run Some Queries

First, lets use step-wise execution and the lower-level API to create and execute a task.

In [ ]:
task = agent.create_task(
    "Given the risk factors of Uber and Lyft described in their 10K files, "
    "which company is performing better? Please use concrete numbers to inform your decision."
)

In [ ]:
# run initial step
step_output = agent.run_step(task.task_id)

> Selecting node to expand: Observation: Given the risk factors of Uber and Lyft described in their 10K files, which company is performing better? Please use concrete numbers to inform your decision.
> Got candidates: ["Review the risk factors detailed in Uber's 10K report.", "Review the risk factors detailed in Lyft's 10K report."]
=== Calling Function ===
Calling function: lyft_10k with args: {"input": "What are the risk factors detailed in Lyft's 10K report for 2021?"}
=== Calling Function ===
Calling function: uber_10k with args: {"input": "What are the risk factors mentioned in Uber's 10K report for 2021?"}
=== Function Output ===
The risk factors mentioned in Uber's 10K report for 2021 include the company's ability to attract or maintain a critical mass of drivers, consumers, merchants, shippers, and carriers. If they fail to attract or retain these key groups, the platform could become less appealing, which would adversely impact Uber's financial results. Another risk factor is 

From the step output, we can inspect the task state

In [ ]:
for step in (
    step_output.task_step.step_state["root_node"].children[0].current_reasoning
):
    print(step)
    print("---------")

observation='Given the risk factors of Uber and Lyft described in their 10K files, which company is performing better? Please use concrete numbers to inform your decision.' return_direct=False
---------
observation="Review the risk factors detailed in Uber's 10K report." return_direct=False
---------


In [ ]:
for step in (
    step_output.task_step.step_state["root_node"]
    .children[0]
    .children[0]
    .current_reasoning
):
    print(step)
    print("---------")

observation='Given the risk factors of Uber and Lyft described in their 10K files, which company is performing better? Please use concrete numbers to inform your decision.' return_direct=False
---------
observation="Review the risk factors detailed in Uber's 10K report." return_direct=False
---------
thought="The user is asking for information on Uber's risk factors from the 10K report for 2021. I need to use the uber_10k tool to retrieve this information." action='uber_10k' action_input={'input': "What are the risk factors mentioned in Uber's 10K report for 2021?"}
---------
observation="The risk factors mentioned in Uber's 10K report for 2021 include the company's ability to attract or maintain a critical mass of drivers, consumers, merchants, shippers, and carriers. If they fail to attract or retain these key groups, the platform could become less appealing, which would adversely impact Uber's financial results. Another risk factor is related to retaining and attracting high-quality

Lets loop until we finish the task

In [ ]:
# repeat until the last step is reached
while not step_output.is_last:
    step_output = agent.run_step(task.task_id)

response = agent.finalize_response(task.task_id)

> Selecting node to expand: Observation: The risk factors mentioned in Uber's 10K report for 2021 include the company's ability to attract or maintain a critical mass of drivers, consumers, merchants, shippers, and carriers. If they fail to attract or retain these key groups, the platform could become less appealing, which would adversely impact Uber's financial results. Another risk factor is related to retaining and attracting high-quality personnel. Continued attrition, future attrition, or unsuccessful succession planning could also adversely affect Uber's business.
> Got candidates: ["Review the risk factors detailed in Lyft's 10K report.", 'Compare the risk factors of Uber and Lyft to determine which company is better managing its risks.']
=== Calling Function ===
Calling function: lyft_10k with args: {"input": "What are the risk factors mentioned in Lyft's 10K report for 2021?"}
=== Calling Function ===
Calling function: lyft_10k with args: {"input": "What are the risk factors m

In [ ]:
print(str(response))

Analyzing the financial summaries from the 2021 10K reports for both Uber and Lyft, we can make several observations:

1. **Revenue Growth**: Uber showed a significant revenue increase from $11,139 million in 2020 to $17,455 million in 2021. Lyft also reported an increase, but on a smaller scale, from $2,364,681,000 in 2020 to $3,208,323,000 in 2021.

2. **Net Loss Improvement**: Both companies reduced their net losses year-over-year. Uber reduced its net loss from $6,768 million in 2020 to $496 million in 2021. Similarly, Lyft reduced its net loss from $1,752,857,000 in 2020 to $1,009,359,000 in 2021.

3. **Operational Efficiency**: Uber's loss from operations decreased substantially from $3,834 million in 2021, compared to Lyft, which reported a loss from operations of $1,082,432,000. Uber's improvement in operating loss as a percentage of revenue is more pronounced than that of Lyft.

Considering these factors, Uber appears to be managing its risks better than Lyft, evidenced by hig

Using `.chat()` directly, we can have the agent automatically run until completion.

In [ ]:
agent.reset()

response = agent.chat(
    "Given the revenue growth and risk factors of Uber and Lyft, "
    "which company is performing better? Please use concrete numbers to inform your decision."
)

> Selecting node to expand: Observation: Given the revenue growth and risk factors of Uber and Lyft, which company is performing better? Please use concrete numbers to inform your decision.
> Got candidates: ['Collect and compare the recent financial data on revenue growth for Uber and Lyft.', 'Identify and evaluate the main risk factors currently affecting Uber and Lyft.']
=== Calling Function ===
Calling function: uber_10k with args: {"input": "What was Uber's revenue growth in 2021?"}
=== Calling Function ===
Calling function: uber_10k with args: {"input": "What was Uber's revenue growth in 2021 and what are the main risk factors affecting the company?"}
=== Function Output ===
Uber's revenue grew by 57% in 2021, increasing from $11,139 million in 2020 to $17,455 million.
=== Function Output ===
Uber's revenue in 2021 was $17.5 billion, marking a 57% increase from the previous year. The main risk factors affecting the company include fluctuations in operating results due to factors 

In [ ]:
print(str(response))

In 2021, Uber's revenue growth rate was 57%, while Lyft's was approximately 35.7%. Based on these figures, Uber experienced a higher revenue growth rate compared to Lyft.
